# Session 1: Introduction to Bioinformatics File Formats

## Learning Objectives

By the end of this session, you will be able to:

1. Identify common bioinformatics file formats and their use cases
2. Work with FASTA and FASTQ files
3. Learn how to install and run your first bioinfomatics tool
4. Understand quality scores in sequencing data
5. Navigate BED and GFF annotation formats
6. Apply best practices for organising bioinformatics projects

## 1. Common Bioinformatics File Formats

### 1.1 Sequence Data Formats

Nucleotide, protein, and reference genome sequences are stored in two plain-text formats widely used in bioinformatics: FASTA and FASTQ. We will discuss both file formats below and explore tools for working with data in these formats.

#### FASTA Format

FASTA is a plain text format and one of the simplest and most widely used formats for storing nucleotide, protein sequences, coding DNA sequences (CDS), transcript sequences, reference genome files, and so on. It originates from the FASTA alignment suite, created by [William R. Pearson](https://fasta.bioch.virginia.edu/wrpearson/) and [David J. Lipman](https://en.wikipedia.org/wiki/David_J._Lipman).

**Structure:**
FASTA files are composed of sequence entries, each containing a description and the sequence data.
- The description line (header line) starts with the greater than symbol `>` followed by a sequence identifier and description
- Sequence data follows on subsequent lines and continues until there is another description line starting with `>`

```
>sequence_id description
ATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG
```

#### FASTQ Format

FASTQ extends FASTA by including quality scores for each base. This is the standard output format for most sequencing data produced by modern high-throughput sequencing platforms. The per-base quality score indicates the confidence for each base call.

**Structure (4 lines per read):**
1. Header line starting with `@` followed by sequence identifier and other information
2. Raw sequence
3. Separator line starting with `+`
4. Quality scores (same length as sequence)

**Example from our training data:**
```bash
# View the first read from the training FASTQ file
gzcat trining_datasets/SRR10532784_1.fastq.gz | head -4
```

```
@SRR10532784.1 1/1
NTTAAAAGTGAGAATAAATAATAAAGATTCTTATATGGAGCAAAGGTTTTTCTTCTTGTATAAACTGAGACTATCACAATTAAAGTTTGGCGTCTGAACTCGTCCTTTAGTAGTGGCAGCTTGTAGGGAAAGAGTTCTGGCTTGTCTCTC
+
#AAFA<J<JJ-<JJA-J<JAFFA<A<<-7<<J<AFJJ<A7AA--FJFFFAFFJ-<<AAA-<---FJJ-F-7AF---AFFJJF-77FAF7AAJJ7-7--7--7-7-777--7--7-<A)7))77----7---77-7---77))-)----7-
```

### Quality Score Reference

You will hear about the quality score of sequenced reads, as different sequencing instruments produce reads with varying quality metrics.

| Phred Score | Error Probability | Accuracy |
|------------|-------------------|----------|
| 10 | 1 in 10 | 90% |
| 20 | 1 in 100 | 99% |
| 30 | 1 in 1,000 | 99.9% |
| 40 | 1 in 10,000 | 99.99% |

**Rule of thumb:** Q30 or higher is generally considered high quality for Illumina data.

**ASCII Encoding:**
Quality scores are encoded as ASCII characters. For Illumina 1.8+ (Phred+33):
- `!` = 0 (worst)
- `I` = 40 (excellent)
- `#` = 2 (very poor, often indicates N base call)


The fastq or fasta files contains million of short/long reads from a high-throughput sequencer where their genomic position, function and consequences relative to the reference genome is not known. This is a result of digesting the nucleic acid during library preperation, where DNA or RNA is cut into a particular size, adapters are added and then loaded to be sequenced. There isn't a genomic sequencer that is capable of sequencing a genome on it's entire length, despite the fact that a good progress has been made with pore sequencing. However, reconstructing reads in respect to a genome reference is still required in a process commpnly known as alignemnt or mapping. 

### 1.2 Alignment Formats

#### SAM/BAM Format

**SAM (Sequence Alignment/Map)** is a text format for storing sequence alignments. **BAM** is the compressed binary version.

Key fields in SAM format:
1. **QNAME**: Query (read) name
2. **FLAG**: Bitwise flag (paired, mapped, strand, etc.)
3. **RNAME**: Reference sequence name
4. **POS**: 1-based leftmost mapping position
5. **MAPQ**: Mapping quality
6. **CIGAR**: Alignment description string
7. **RNEXT**: Reference name of mate/next read
8. **PNEXT**: Position of mate/next read
9. **TLEN**: Template length
10. **SEQ**: Segment sequence
11. **QUAL**: Quality scores

#### CRAM Format

CRAM is a reference-based compressed columnar format for storing sequence alignments, developed as a more space-efficient alternative to BAM. It achieves 30-60% smaller file sizes through reference-based compression and advanced encoding schemes.

**Key Differences from BAM**
**Reference dependency:** CRAM requires the reference genome (.fasta) for both reading and writing, as it stores only differences from the reference rather than full sequences.
Compression modes: Supports both lossless (preserving all data) and lossy compression (discarding quality scores or other fields for additional space savings).

**Storage architecture:** Uses columnar storage organized into containers and slices, rather than sequential read-by-read storage like SAM/BAM.

**CRAM File Contents**

CRAM files contain the same alignment information as SAM/BAM but organized differently:

**File header:** SAM-compatible header including reference sequences, read groups, and program information, plus reference genome MD5 checksum for validation.

**Containers and slices:** Data organized into hierarchical containers (units of compression) containing slices (subsets of reads from specific genomic regions).

**Alignment fields:** Same core fields as SAM/BAM:
QNAME, FLAG, RNAME, POS, MAPQ, CIGAR
RNEXT, PNEXT, TLEN
SEQ (stored as differences from reference)
QUAL (optionally preserved or discarded)
Optional tags

**Compression codecs:** Multiple algorithms (gzip, bzip2, lzma, rANS) applied per field type for optimal compression.



**Example from our training data:**

In order to view or manipualte the alignment file we need to use a specialised bioinfomatics tools developed for interacting and post processing of genomic DNA aligments in the SAM, BAM and CRAM format. [Samtools](https://www.htslib.org/) is a suite of programs for interacting with high-throughput sequencing read alignments. 

**However,before installing our first bioinfomatics tool let's learn how we can actually install it**

Generally, if you are working with a high preformance computing (HPC) to analyse youe bioinformatics data your system admin will be helping you with installing required bioinfomatics tools as most of them require a set of dependencies to be installed first. However, the bioinformatics comunity has made a use of a package managers such as [conda](https://anaconda.org/anaconda/conda) and containeried applications hosted in [Docker](https://www.docker.com/). This has saved bioinfomaticins immense mount of time spent on installing deppendecies and getting tools to work. 

**Let's install samtools in our machines** 

First of all you need to install mini-conda (a light weight miniature installation of Anaconda) by following the [instructions](https://www.anaconda.com/docs/getting-started/miniconda/install#windows-command-prompt)  

After instailling miniconda install samtools globally using cond and view the provided bam file available in training.

```bash 
# Installing samtools 
conda install bioconda::samtools

# Investigate samtools command line options 
samtools 

# View alignments from the sheep BAM file
samtools view trining_datasets/u9_liver_100.bam | head -2
```

#### Understanding CIGAR Strings

The CIGAR string describes how a read aligns to the reference. Each operation is represented by a number followed by a letter:

| Operation | Description | Consumes Query | Consumes Reference |
|-----------|-------------|----------------|--------------------|
| M | Match or mismatch | Yes | Yes |
| I | Insertion to reference | Yes | No |
| D | Deletion from reference | No | Yes |
| N | Skipped region (intron) | No | Yes |
| S | Soft clipping | Yes | No |
| H | Hard clipping | No | No |

**Examples:**
- `50M` - 50 bases match/mismatch
- `30M2I18M` - 30 matches, 2 base insertion, 18 matches
- `25M100N25M` - 25 matches, 100 base intron (RNA-seq), 25 matches
- `16S109M` - 16 bases soft clipped, 109 matches (seen in our BAM file)

#### BAM vs CRAM: Choosing the Right Format

| Feature | BAM | CRAM |
|---------|-----|------|
| Compression | ~25% of SAM | ~50% of BAM |
| Speed | Faster | Slower |
| Reference required | No | Yes (for full compression) |
| Tool support | Universal | Growing |
| Best for | Active analysis | Long-term storage |

**Recommendation:** Use BAM during active analysis, convert to CRAM for archival storage.

```bash
# Convert BAM to CRAM
samtools view -T reference.fasta -C -o output.cram input.bam

# Convert CRAM back to BAM
samtools view -T reference.fasta -b -o output.bam input.cram
```

#### Working with SAM/BAM Files

**samtools** is the standard tool for manipulating alignment files: Here are common command line tools for using samtools

```bash
# View BAM file header
samtools view -H trining_datasets/u9_liver_100.bam

# Convert SAM to BAM
samtools view -bS aligned.sam > aligned.bam

# Sort BAM file by coordinate
samtools sort aligned.bam -o aligned.sorted.bam

# Index sorted BAM file (required for most downstream tools)
samtools index aligned.sorted.bam

# View alignments in a specific region
samtools view aligned.sorted.bam chr1:1000000-2000000

# Get alignment statistics
samtools flagstat trining_datasets/u9_liver_100.bam

# Calculate coverage depth
samtools depth aligned.sorted.bam > coverage.txt
```

### 1.3 Variant Formats

#### VCF (Variant Call Format)

VCF is the standard format for storing genetic variant data. It is used by virtually all variant calling pipelines.

**Structure:**
- Meta-information lines (starting with `##`)
- Header line (starting with `#`)
- Data lines (one variant per line)

**Required columns:**
1. CHROM - Chromosome
2. POS - Position (1-based)
3. ID - Variant identifier (e.g., rs number)
4. REF - Reference allele
5. ALT - Alternative allele(s)
6. QUAL - Phred-scaled quality score
7. FILTER - Filter status (PASS or failed filter names)
8. INFO - Additional information (key=value pairs)

**Example from our training data:**
```bash
# View VCF header
bcftools view -h trining_datasets/sheep.snp.vcf.gz | head -20

# View first few variants
bcftools view -H trining_datasets/sheep.snp.vcf.gz | head -3
```

#### Understanding Genotypes in VCF

The genotype field (GT) uses numbers to represent alleles:

| Genotype | Meaning | Description |
|----------|---------|-------------|
| 0/0 | Homozygous reference | Both alleles match reference |
| 0/1 | Heterozygous | One reference, one alternate |
| 1/1 | Homozygous alternate | Both alleles are alternate |
| 1/2 | Heterozygous (multi-allelic) | Two different alternate alleles |
| ./. | Missing | Genotype could not be determined |

**Phased vs Unphased:**
- `0/1` - Unphased (we don't know which chromosome has which allele)
- `0|1` - Phased (first allele is on the maternal/paternal chromosome)

Our sheep VCF contains ~300 samples with genotypes for each variant.

#### Working with VCF Files

**bcftools** is the standard tool for VCF manipulation. [BCFtools](https://samtools.github.io/bcftools/bcftools.html) contains a set of utilities to manipulate variant clalling format (VCF):

```bash

# Installing bcftools 
conda install bioconda::samtools

# View VCF file
bcftools view trining_datasets/sheep.snp.vcf.gz | less

# Filter variants by quality
bcftools filter -i 'QUAL>30' trining_datasets/sheep.snp.vcf.gz > filtered.vcf

# Extract only SNPs (already SNP-only in our file)
bcftools view -v snps variants.vcf.gz > snps_only.vcf

# Get variant statistics
bcftools stats trining_datasets/sheep.snp.vcf.gz > stats.txt

# Count variants per chromosome
bcftools view -H trining_datasets/sheep.snp.vcf.gz | cut -f1 | sort | uniq -c

# Extract specific region
bcftools view trining_datasets/sheep.snp.vcf.gz chr1:1-1000 > region.vcf
```

### 1.4 Annotation Formats

#### BED Format (Browser Extensible Data)

BED is a tab-delimited format for describing genomic intervals. It is simpler than GFF/GTF and commonly used for:
- Peak calling results (ChIP-seq, ATAC-seq)
- Target regions for capture sequencing
- Genomic features and intervals

**Important:** BED uses 0-based, half-open coordinates (start is 0-based, end is excluded).

**Standard BED columns:**

| Column | Name | Description |
|--------|------|-------------|
| 1 | chrom | Chromosome name |
| 2 | chromStart | Start position (0-based) |
| 3 | chromEnd | End position (excluded) |
| 4 | name | Feature name (optional) |
| 5 | score | Score 0-1000 (optional) |
| 6 | strand | + or - (optional) |

**Example:**
```
chr1    0       100     feature1    500     +
chr1    200     300     feature2    750     -
chr2    1000    2000    feature3    1000    +
```

#### GFF/GTF Format (Gene Feature Format)

GFF (General Feature Format) and GTF (Gene Transfer Format) are used for gene annotations. GTF is a stricter version of GFF2.

**Important:** GFF/GTF uses 1-based, fully-closed coordinates (both start and end are included).

**GFF3 columns:**

| Column | Name | Description |
|--------|------|-------------|
| 1 | seqid | Chromosome/scaffold name |
| 2 | source | Program or database |
| 3 | type | Feature type (gene, exon, CDS, etc.) |
| 4 | start | Start position (1-based) |
| 5 | end | End position (included) |
| 6 | score | Score or "." |
| 7 | strand | + or - |
| 8 | phase | Reading frame (0, 1, 2) or "." |
| 9 | attributes | Key=value pairs |

#### Coordinate System Comparison

Understanding coordinate systems is crucial to avoid off-by-one errors:

| Format | Start | End | Example (bases 1-10) |
|--------|-------|-----|---------------------|
| BED | 0-based | excluded | 0-10 |
| GFF/GTF | 1-based | included | 1-10 |
| SAM | 1-based | included | 1-10 |
| VCF | 1-based | included | 1-10 |

**Conversion:**
- BED to GFF: start + 1, end stays the same
- GFF to BED: start - 1, end stays the same

## 2. File Format Summary Table

| Format | Extension | Type | Coordinates | Use Case | Compressed Version |
|--------|-----------|------|-------------|----------|-------------------|
| FASTA | .fa, .fasta | Text | N/A | Reference sequences | .fa.gz |
| FASTQ | .fq, .fastq | Text | N/A | Raw sequencing reads | .fq.gz |
| SAM | .sam | Text | 1-based | Sequence alignments | - |
| BAM | .bam | Binary | 1-based | Sequence alignments | Already compressed |
| CRAM | .cram | Binary | 1-based | Sequence alignments | Higher compression |
| VCF | .vcf | Text | 1-based | Variant calls | .vcf.gz |
| BCF | .bcf | Binary | 1-based | Variant calls | Already compressed |
| BED | .bed | Text | 0-based | Genomic intervals | .bed.gz |
| GFF/GTF | .gff, .gtf | Text | 1-based | Gene annotations | .gff.gz |

## 3. Hands-On Exercise

Using the training datasets provided, complete the following tasks:

### Exercise 1: Exploring the FASTQ File

```bash
# 1. Count the total number of reads in the FASTQ file
# Hint: Each read has 4 lines, so divide total lines by 4
gzcat trining_datasets/SRR10532784_1.fastq.gz | wc -l
# Then divide by 4

# 2. Extract the first 10 read identifiers
gzcat trining_datasets/SRR10532784_1.fastq.gz | grep "^@SRR" | head -10

# 3. Find reads that start with 'N' (unknown base)
gzcat trining_datasets/SRR10532784_1.fastq.gz | awk 'NR%4==2' | grep "^N" | wc -l
```

### Exercise 2: Exploring the BAM File

```bash
# 1. View the BAM header to see which chromosomes are present
samtools view -H trining_datasets/u9_liver_100.bam | grep "^@SQ"

# 2. Count total alignments
samtools view -c trining_datasets/u9_liver_100.bam

# 3. Count mapped reads only (exclude unmapped)
samtools view -c -F 4 trining_datasets/u9_liver_100.bam

# 4. Get alignment statistics
samtools flagstat trining_datasets/u9_liver_100.bam
```

### Exercise 3: Exploring the VCF File

```bash
# 1. Count total number of variants
bcftools view -H trining_datasets/sheep.snp.vcf.gz | wc -l

# 2. Count variants on chromosome 1
bcftools view -H trining_datasets/sheep.snp.vcf.gz chr1 | wc -l

# 3. Find how many samples are in the VCF
bcftools query -l trining_datasets/sheep.snp.vcf.gz | wc -l

# 4. Extract variants with QUAL > 50000
bcftools view -i 'QUAL>50000' trining_datasets/sheep.snp.vcf.gz | bcftools view -H | wc -l
```

## Additional Resources

- [SAM Format Specification](https://samtools.github.io/hts-specs/SAMv1.pdf)

- [VCF Format Specification](https://samtools.github.io/hts-specs/VCFv4.3.pdf)

- [BED Format (UCSC)](https://genome.ucsc.edu/FAQ/FAQformat.html#format1)

- [GFF3 Specification](https://github.com/The-Sequence-Ontology/Specifications/blob/master/gff3.md)

- [FASTQ Format Wikipedia](https://en.wikipedia.org/wiki/FASTQ_format)

- [Bioinformatics Data Skills (Book)](https://www.oreilly.com/library/view/bioinformatics-data-skills/9781449367480/)

- [Pearson WR, Lipman DJ. Improved tools for biological sequence comparison. Proc Natl Acad Sci U S A. 1988 Apr;85(8):2444-8. doi: 10.1073/pnas.85.8.2444. PMID: 3162770; PMCID: PMC280013.](https://pubmed.ncbi.nlm.nih.gov/3162770/#)

- [Heng Li, Bob Handsaker, Alec Wysoker, Tim Fennell, Jue Ruan, Nils Homer, Gabor Marth, Goncalo Abecasis, Richard Durbin, 1000 Genome Project Data Processing Subgroup, The Sequence Alignment/Map format and SAMtools, Bioinformatics, Volume 25, Issue 16, August 2009, Pages 2078–2079, https://doi.org/10.1093/bioinformatics/btp352](https://academic.oup.com/bioinformatics/article/25/16/2078/204688)
